In [1]:
import sys
from tqdm import tqdm
sys.path.insert(0, '../')

import torch
from torch import nn
from torch.nn import functional as F

from attention import MultiHeadAttention

In [21]:
tensor = torch.rand(4,4)
tensor

tensor([[0.6499, 0.3315, 0.4814, 0.5345],
        [0.8124, 0.6533, 0.0796, 0.6233],
        [0.3390, 0.0468, 0.3367, 0.3246],
        [0.4309, 0.1613, 0.2276, 0.6811]])

In [26]:
booltensor = torch.BoolTensor(4,4)

In [47]:
tensor.tril(diagonal=0)

tensor([[0.6499, 0.0000, 0.0000, 0.0000],
        [0.8124, 0.6533, 0.0000, 0.0000],
        [0.3390, 0.0468, 0.3367, 0.0000],
        [0.4309, 0.1613, 0.2276, 0.6811]])

In [39]:
INF = -1e12

In [43]:
F.softmax(tensor.masked_fill(mask=booltensor, value=INF), dim=1)

tensor([[0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.2500, 0.2500, 0.2500, 0.2500],
        [0.0000, 0.0000, 0.3885, 0.6115]])

In [9]:
torch.tril(tensor, diagonal=0)

tensor([[0.8779, 0.0000, 0.0000, 0.0000],
        [0.4482, 0.2513, 0.0000, 0.0000],
        [0.7568, 0.9910, 0.7608, 0.0000],
        [0.0457, 0.4831, 0.8826, 0.8791]])

In [12]:
pad_id = 0
vocab_size = 100
max_len = 10
hidden_dim = 6

def padding(data: list, pad_id: int=0) -> (list, int):
    max_len = len(max(data, key=len))
    output = [sample + [pad_id]*(max_len-len(sample)) for sample in tqdm(data)]
    return output, max_len

data = [
     [62, 13, 47, 39, 78, 33, 56, 13, 39, 29],
     [60, 96, 51, 32, 90, 44, 86, 71, 36, 18],
     [35, 45, 48, 65, 91, 99, 92, 10, 31, 21],
     [75, 51, 45, 48, 65, 91, 99, 11, 13, 28],
     [66, 88, 98, 47, 48, 65, 17, 13, 67, 99],
     ]

# data, max_len = padding(data)
data = torch.LongTensor(data)
print(data.shape)

torch.Size([5, 10])


In [13]:
embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=hidden_dim)
embedding.weight.data.uniform_(-1, 1)
print('Weight initialized')

Weight initialized


In [14]:
X_embedded = embedding(data)
X_embedded.size()

torch.Size([5, 10, 6])

In [15]:
w_query = nn.Linear(in_features=hidden_dim, out_features=hidden_dim) # Query
w_key = nn.Linear(in_features=hidden_dim, out_features=hidden_dim) # Key
w_value = nn.Linear(in_features=hidden_dim, out_features=hidden_dim) # Value

query = w_query(X_embedded)
key = w_key(X_embedded)
value = w_value(X_embedded)

In [19]:
attention = MultiHeadAttention(hidden_dim=hidden_dim, num_heads=2)
attention_mat = attention(query, key, value)

In [21]:
attention_mat.size()

torch.Size([5, 10, 6])

In [26]:
attn_layer_norm = nn.LayerNorm(normalized_shape=(10, 6))

In [25]:
attention_mat.size()[1:]

torch.Size([10, 6])

In [27]:
attn_layer_norm(attention_mat).size()

torch.Size([5, 10, 6])